In [1]:
import pandas as pd
import numpy as np

import misc_funcs
import data

import csv

from multiprocess import Pool,cpu_count
from itertools import product

In [7]:
#No need to run this again. The output is saved in conflict_positions.

#misc_funcs.conflict_position("battles")

In [9]:
#No need to run this again. The output is saved in event_mappings.

#a = misc_funcs.conflict_event_polygon_mapping(640,"battles",29)

In [14]:
#No need to run this again. The output is saved in 80/160/320/640.

#misc_funcs.single_tile_events(640,"battles")

In [13]:
#No need to run this again. The output is saved in FG_time_series.

#dx = 640
#ts = misc_funcs.time_series_all_polygons(1,dx,"battles")
#ts.to_csv(f"generated_data/battles/FG_time_series/time_series_1_{str(dx)}.csv" , index=False)

In [2]:
ts = misc_funcs.time_series_all_polygons(64,320,"battles")
a,b = misc_funcs.avalanche_creation_fast_st(ts,64,320,"battles")

Creating time bins!
Done!


/home/niraj/armed_conflict/misc_funcs.py:261: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  neighbors_arr = np.array(neighbors_list)


In [3]:
len(a)

4356

In [2]:
a,b,c = misc_funcs.avalanche_creation_fast_te(64,80,"battles","data")

Calculating transfer entropy between tiles!
Shuffle Number: 0
Shuffle Number: 10
Shuffle Number: 20
Shuffle Number: 30
Shuffle Number: 40
Shuffle Number: 50
Done!
Calculating self loop entropies!
Shuffle Number: 0
Shuffle Number: 10
Shuffle Number: 20
Shuffle Number: 30
Shuffle Number: 40
Shuffle Number: 50


In [3]:
len(a)

4052

# Generating avalanches

In [3]:
def loop_wrapper_st(args):
    dx , time = args
    ts = misc_funcs.time_series_all_polygons(time,dx,"battles")
    a,b = misc_funcs.avalanche_creation_fast_st(ts,time,dx,"battles")
    return a

In [2]:
def loop_wrapper_te(args):
    dx , time = args
    a,b,c = misc_funcs.avalanche_creation_fast_te(time,dx,"battles","data")
    return a

In [5]:
with Pool(18) as pool:
    output = list(pool.map(loop_wrapper_te , product([80,160,320,640],[2,4,8,16,32,64,128,256,512])))

Calculating transfer entropy between tiles!
Shuffle Number: 0
Calculating transfer entropy between tiles!
Shuffle Number: 0
Calculating transfer entropy between tiles!
Shuffle Number: 0
Calculating transfer entropy between tiles!
Shuffle Number: 0
Calculating transfer entropy between tiles!
Shuffle Number: 0
Calculating transfer entropy between tiles!
Calculating transfer entropy between tiles!
Shuffle Number: 0
Calculating transfer entropy between tiles!
Calculating transfer entropy between tiles!
Shuffle Number: 0
Shuffle Number: 0
Shuffle Number: 0
Calculating transfer entropy between tiles!
Shuffle Number: 0
Calculating transfer entropy between tiles!
Shuffle Number: 0
Calculating transfer entropy between tiles!
Calculating transfer entropy between tiles!
Shuffle Number: 0
Shuffle Number: 0
Calculating transfer entropy between tiles!
Shuffle Number: 0
Calculating transfer entropy between tiles!
Shuffle Number: 0
Calculating transfer entropy between tiles!
Shuffle Number: 0
Calculat

In [6]:
dx_time = list(product([80,160,320,640],[2,4,8,16,32,64,128,256,512]))

In [7]:
for i in range(len(dx_time)):
    ava_list = output[i]
    
    with open(f"avalanches/battles/te/te_ava_box_{dx_time[i][1]}_{dx_time[i][0]}.csv","w") as f:
        wr = csv.writer(f)
        wr.writerows(output[i])

## Box to Event ava conversion

In [17]:
type_of_algo = "st"

def box_to_event(args):
    dx , time = args
    
    box_path = f"avalanches/battles/{type_of_algo}/{type_of_algo}_ava_box_{str(time)}_{str(dx)}.csv"
    box_ava = misc_funcs.extract_box_ava_from_file(box_path)
    
    ts_FG = pd.read_csv(f"generated_data/battles/FG_time_series/time_series_1_{str(dx)}.csv")
    data_bin = misc_funcs.data_bin_extracter(ts_FG,time)
    
    event_ava = misc_funcs.boxAva_to_eventAva(box_ava,data_bin)
    
    return event_ava

In [18]:
with Pool(18) as pool:
    output = list(pool.map(box_to_event , product([80,160,320,640],[2,4,8,16,32,64,128,256,512])))

In [19]:
dx_time = list(product([80,160,320,640],[2,4,8,16,32,64,128,256,512]))

In [20]:
for i in range(len(dx_time)):
    ava_list = output[i]
    
    with open(f"avalanches/battles/{type_of_algo}/{type_of_algo}_ava_event_{dx_time[i][1]}_{dx_time[i][0]}.csv","w") as f:
        wr = csv.writer(f)
        wr.writerows(output[i])

## Analysis

In [4]:
time = 64
dx = 160
type_of_algo = "te"
conflict_type = "battles"

In [3]:
event_path = f"avalanches/battles/{type_of_algo}/{type_of_algo}_ava_event_{str(time)}_{str(dx)}.csv"

In [5]:
a = misc_funcs.ava_numbering(time,dx,conflict_type,event_path)

Creating final data table!
Creating time bins!
Done!


/home/niraj/anaconda3/envs/armed_conflict/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Done!


In [6]:
a

,polygon_number,date,days,bins,avalanche_number
0,340.0,01 January 1997,0,1,1
1,1793.0,01 January 1997,0,1,2
2,1631.0,01 January 1997,0,1,20
3,1556.0,01 January 1997,0,1,30
4,1792.0,01 January 1997,0,1,38
...,...,...,...,...,...
58875,2321.0,17 July 2020,8598,135,0
58876,1052.0,17 July 2020,8598,135,0
58877,2170.0,17 July 2020,8598,135,0
58878,565.0,18 July 2020,8599,135,0
